In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yahooquery as yq
from datetime import datetime
import time
import traceback

# Show all columns
pd.set_option('display.max_columns', None)

In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]["Symbol"]
display(tickers) # Show all tickers

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [3]:
test = tickers.loc[tickers == 'BAC']
test

59    BAC
Name: Symbol, dtype: object

In [13]:
yearly_combined = pd.DataFrame()

for ticker in tickers:
    start = time.time()
    try:
        stock = yq.Ticker(ticker)
        
        SumDetail = stock.summary_detail
        info = stock.summary_profile
        
        balance_sheet = stock.balance_sheet()
        # Fetch the income statement
        income_statement = stock.income_statement()


        cashflow = stock.cash_flow()

        if isinstance(balance_sheet, pd.DataFrame) & isinstance(income_statement, pd.DataFrame) & isinstance(cashflow, pd.DataFrame):

            start_dates = balance_sheet['asOfDate'].values
            end_dates = start_dates + np.timedelta64(366, 'D')

            start_dates = [str(date)[:-19] for date in start_dates]
            end_dates = [str(date)[:-19] for date in end_dates]

            year1 = stock.history(start=start_dates[0], end=end_dates[0], period= 'day').reset_index(drop=True)
            year2 = stock.history(start=start_dates[1], end=end_dates[1], period= 'day').reset_index(drop=True)
            year3 = stock.history(start=start_dates[2], end=end_dates[2], period= 'day').reset_index(drop=True)
            year4 = stock.history(start=start_dates[3], end=end_dates[3], period= 'day').reset_index(drop=True)

            columns = [2019, 2020, 2021, 2022]

            balance_sheet = balance_sheet.set_index('asOfDate')
            balance_sheet = balance_sheet.T
            income_statement = income_statement.set_index('asOfDate')
            income_statement = income_statement.T
            cashflow = cashflow.set_index('asOfDate')
            cashflow = cashflow.T

            # Remove duplicate columns from all
            balance_sheet = balance_sheet.loc[:,~balance_sheet.columns.duplicated()]
            income_statement = income_statement.loc[:,~income_statement.columns.duplicated()]
            cashflow = cashflow.loc[:,~cashflow.columns.duplicated()]
            
            # if number of columns is greater than 4 remove the extra columns
            if len(balance_sheet.columns) > 4:
                balance_sheet = balance_sheet.iloc[:, :4]
            if len(income_statement.columns) > 4:
                income_statement = income_statement.iloc[:, :4]
            if len(cashflow.columns) > 4:
                cashflow = cashflow.iloc[:, :4]
            
            # Rename the columns
            balance_sheet.columns = columns
            income_statement.columns = columns
            cashflow.columns = columns
            cashflow = cashflow.drop(['NetIncome'], axis=0)

            yearly = pd.concat([balance_sheet, income_statement, cashflow]).T
            columns = yearly.columns.values
            # Drop duplicate columns
            yearly = yearly.loc[:,~yearly.columns.duplicated()]

            # Calculate the ratios

            # Liquidity Ratios
            if 'CurrentAssets' not in columns:
                if 'CashCashEquivalentsAndShortTermInvestments' not in columns:
                    yearly['CurrentAssets'] = yearly['CashAndCashEquivalents']
                else:
                    yearly['CurrentAssets'] = yearly['CashCashEquivalentsAndShortTermInvestments']
            if 'CurrentLiabilities' not in columns:
                yearly['CurrentLiabilities'] = yearly['TotalLiabilitiesNetMinorityInterest'] - yearly['LongTermDebt']
            yearly['CurrentRatio'] = yearly['CurrentAssets'] / yearly['CurrentLiabilities']
            yearly['QuickRatio'] = yearly['CurrentAssets']
            if 'Inventory' in columns:
                yearly['QuickRatio'] += yearly['Inventory']
            yearly['QuickRatio'] /= yearly['CurrentLiabilities']
            yearly['CashRatio'] = yearly['CashAndCashEquivalents'] / yearly['CurrentLiabilities']
            if 'CashAndCashEquivalentsAndShortTermInvestments' in columns:
                yearly['CashRatio'] = yearly['CashAndCashEquivalentsAndShortTermInvestments'] / yearly['CurrentLiabilities']
            else:
                yearly['CashRatio'] = yearly['CashAndCashEquivalents'] / yearly['CurrentLiabilities']
            yearly['OperatingCashFlowRatio'] = yearly['OperatingCashFlow'] / yearly['CurrentLiabilities']
            yearly['equityRatio'] = yearly['StockholdersEquity'] / yearly['TotalAssets']

            # Leverage
            yearly['DebtToEquity'] = yearly['TotalLiabilitiesNetMinorityInterest'] / yearly['StockholdersEquity']
            yearly['DebtToAssets'] = yearly['TotalLiabilitiesNetMinorityInterest'] / yearly['TotalAssets']

            if 'InterestExpense' not in columns:
                yearly['InterestCoverage'] = np.nan
            elif ('OperatingIncome' not in columns) & ('EBIT' in columns):
                yearly['InterestCoverage'] = yearly['EBIT'] / yearly['InterestExpense']
            elif 'OperatingIncome' in columns:
                yearly['InterestCoverage'] = yearly['OperatingIncome'] / yearly['InterestExpense']
            elif 'OperatingRevenue' in columns:
                yearly['InterestCoverage'] = (yearly['OperatingRevenue'] - (yearly['TotalRevenue'] - yearly['PretaxIncome'] + yearly['InterestExpense'])) / yearly['InterestExpense']
            else:
                yearly['InterestCoverage'] = np.nan

            # Efficiency
            yearly['AssetTurnover'] = yearly['TotalRevenue'] / yearly['TotalAssets']
            if 'Receivables' in columns:
                yearly['ReceivablesTurnover'] = yearly['TotalRevenue'] / yearly['Receivables']
            else:
                yearly['ReceivablesTurnover'] = np.nan
            
            # Profitability
            if ('GrossProfit' not in columns) & ('TotalExpenses' in columns):
                yearly['GrossProfit'] = yearly['TotalRevenue'] - yearly['TotalExpenses']
            elif ('GrossProfit' not in columns) & ('TotalExpenses' not in columns) & ('SellingAndMarketingExpense' in columns):
                yearly['GrossProfit'] = yearly['TotalRevenue'] - (yearly['GeneralAndAdministrativeExpense'] + yearly['InterestExpense'] + 
                                                                yearly['SalariesAndWages'] + yearly['SellingAndMarketingExpense'] + yearly['SellingGeneralAndAdministration'])
            elif ('GrossProfit' not in columns) & ('TotalExpenses' not in columns) & ('SellingAndMarketingExpense' not in columns):
                yearly['GrossProfit'] = yearly['TotalRevenue'] - (yearly['GeneralAndAdministrativeExpense'] + yearly['InterestExpense'])
            yearly['GrossMargin'] = yearly['GrossProfit'] / yearly['TotalRevenue']
            yearly['GrossProfitMargin'] = yearly['NetIncome'] / yearly['TotalRevenue']
            if 'OperatingIncome' not in columns:
                if 'EBIT' not in columns:
                    yearly['EBIT'] = (yearly['OperatingRevenue'] - (yearly['TotalRevenue'] - yearly['PretaxIncome'] + yearly['InterestExpense']))
                yearly['OperatingMargin'] = yearly['EBIT'] / yearly['TotalRevenue']
            else:
                yearly['OperatingMargin'] = yearly['OperatingIncome'] / yearly['TotalRevenue']
            yearly['ReturnOnEquity'] = yearly['NetIncome'] / yearly['StockholdersEquity']
            yearly['ReturnOnAssets'] = yearly['NetIncome'] / yearly['TotalAssets']

            shareprice = np.array([year4.loc[0, 'close'], year3.loc[0, 'close'], year2.loc[0, 'close'], year1.loc[0, 'close']])
            yearly['EPS'] = yearly['NetIncome'] / yearly['BasicAverageShares']
            yearly['PE'] = shareprice / yearly['EPS']

            yearly['Ticker'] = ticker
            yearly['Sector'] = info[ticker]['sector']
            yearly['year'] = yearly.index.values

            startprice = np.array([year4.loc[1, 'open'], year3.loc[1, 'open'], year2.loc[1, 'open'], year1.loc[1, 'open']])
            endprice = np.array([year4.loc[len(year4)-1, 'close'], year3.loc[len(year3)-1, 'close'], year2.loc[len(year2)-1, 'close'], year1.loc[len(year1)-1, 'close']])

            pctDiff = (endprice - startprice) / startprice
            yearly['Result'] = pctDiff

            columns = ['Ticker', 'year', 'Sector', 'CurrentRatio', 'QuickRatio', 'CashRatio', 
                    'OperatingCashFlowRatio', 'equityRatio', 'DebtToEquity', 'DebtToAssets',
                    'InterestCoverage', 'AssetTurnover', 'ReceivablesTurnover', 'GrossMargin',
                    'GrossProfitMargin', 'OperatingMargin', 'ReturnOnEquity', 'ReturnOnAssets',
                    'EPS', 'PE', 'Result']
            
            yearly = yearly[columns].reset_index(drop=True)

            if yearly_combined.empty:
                yearly_combined = yearly
            else:
                yearly_combined = pd.concat([yearly_combined, yearly]).reset_index(drop=True)

            print(ticker, 'done')
        else:
            print(ticker, 'no data')


    except Exception as e:
        print(ticker, e)
        traceback.print_exc()

    time_diff = time.time() - start
    if time_diff < 10:
        time.sleep(10-time_diff)

display(yearly_combined)
yearly_combined.to_csv('SPData.csv', index=False)

MMM done
AOS done
ABT done
ABBV done
ACN done
ATVI done
ADM done
ADBE done
ADP done
AAP done
AES done
AFL done
A done
APD done
AKAM done
ALK done
ALB done
ARE done
ALGN done
ALLE done
LNT done
ALL done
GOOGL done
GOOG done
MO done
AMZN done
AMCR done
AMD done
AEE done
AAL done
AEP done
AXP done
AIG done
AMT done
AWK done
AMP done
ABC done
AME done
AMGN done
APH done
ADI done
ANSS done
AON done
APA done
AAPL done
AMAT done
APTV done
ACGL done
ANET float division by zero


Traceback (most recent call last):
  File "C:\Users\Demor\AppData\Local\Temp\ipykernel_14236\2144300583.py", line 96, in <module>
    yearly['InterestCoverage'] = yearly['OperatingIncome'] / yearly['InterestExpense']
                                 ~~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~
  File "d:\School\Python\Lib\site-packages\pandas\core\ops\common.py", line 72, in new_method
    return method(self, other)
           ^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\arraylike.py", line 126, in __truediv__
    return self._arith_method(other, operator.truediv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\series.py", line 6259, in _arith_method
    return base.IndexOpsMixin._arith_method(self, other, op)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\base.py", line 1325, in _arith_method
    result = ops.arithmet

AJG done
AIZ done
T done
ATO done
ADSK done
AZO done
AVB done
AVY done
BKR done
BALL done
BAC done
BBWI done
BAX done
BDX done
WRB done
BRK.B no data
BBY done
BIO done
TECH done
BIIB done
BLK done
BK done
BA done
BKNG done
BWA done
BXP done
BSX done
BMY done
AVGO done
BR done
BRO done
BF.B no data
BG done
CHRW done
CDNS done
CZR done
CPT done
CPB done
COF done
CAH done
KMX list index out of range


Traceback (most recent call last):
  File "C:\Users\Demor\AppData\Local\Temp\ipykernel_14236\2144300583.py", line 29, in <module>
    year4 = stock.history(start=start_dates[3], end=end_dates[3], period= 'day').reset_index(drop=True)
                                ~~~~~~~~~~~^^^
IndexError: list index out of range


CCL done
CARR done
CTLT done
CAT done
CBOE done
CBRE done
CDW done
CE done
CNC done
CNP done
CDAY done
CF done
CRL done
SCHW done
CHTR done
CVX done
CMG done
CB done
CHD done
CI done
CINF done
CTAS done
CSCO done
C done
CFG done
CLX done
CME done
CMS done
KO done
CTSH done
CL done
CMCSA done
CMA done
CAG done
COP done
ED done
STZ list index out of range


...

Traceback (most recent call last):
  File "C:\Users\Demor\AppData\Local\Temp\ipykernel_14236\2144300583.py", line 105, in <module>
    yearly['ReceivablesTurnover'] = yearly['TotalRevenue'] / yearly['Receivables']
                                    ~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~
  File "d:\School\Python\Lib\site-packages\pandas\core\ops\common.py", line 72, in new_method
    return method(self, other)
           ^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\arraylike.py", line 126, in __truediv__
    return self._arith_method(other, operator.truediv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\series.py", line 6259, in _arith_method
    return base.IndexOpsMixin._arith_method(self, other, op)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\School\Python\Lib\site-packages\pandas\core\base.py", line 1325, in _arith_method
    result = ops.arithmetic_op(l

TT done
TDG done
TRV done
TRMB done
TFC done
TYL done
TSN done
USB done
UDR done
ULTA list index out of range


Traceback (most recent call last):
  File "C:\Users\Demor\AppData\Local\Temp\ipykernel_14236\2144300583.py", line 29, in <module>
    year4 = stock.history(start=start_dates[3], end=end_dates[3], period= 'day').reset_index(drop=True)
                                ~~~~~~~~~~~^^^
IndexError: list index out of range


UNP done
UAL done
UPS done
URI done
UNH done
UHS done
VLO done
VTR done
VRSN done
VRSK done
VZ done
VRTX done
VFC done
VTRS done
VICI done
V done
VMC done
WAB done
WBA done
WMT done
WBD done
WM done
WAT done
WEC done
WFC done
WELL done
WST done
WDC done
WRK done
WY done
WHR done
WMB done
WTW done
GWW done
WYNN done
XEL done
XYL done
YUM done
ZBRA done
ZBH done
ZION done
ZTS done


,Ticker,year,Sector,CurrentRatio,QuickRatio,CashRatio,OperatingCashFlowRatio,equityRatio,DebtToEquity,DebtToAssets,InterestCoverage,AssetTurnover,ReceivablesTurnover,GrossMargin,GrossProfitMargin,OperatingMargin,ReturnOnEquity,ReturnOnAssets,EPS,PE,Result
0,MMM,2019,Industrials,1.406528,1.854804,0.255151,0.766645,0.22533,3.43168,0.77326,13.678571,0.719586,6.475116,0.466766,0.142208,0.19069,0.454139,0.102331,7.920277,15.462843,-0.173409
1,MMM,2020,Industrials,1.885003,2.418344,0.58304,1.02076,0.271777,2.674516,0.726871,12.89603,0.67979,6.663354,0.48406,0.167288,0.211969,0.418435,0.113721,9.32133,19.056295,-0.327501
2,MMM,2021,Industrials,1.704815,2.256558,0.505147,0.825014,0.319638,2.12382,0.678854,15.709016,0.751083,7.41195,0.468392,0.167473,0.216829,0.393527,0.125786,10.226252,17.092283,0.015029
3,MMM,2022,Industrials,1.542371,2.106479,0.383808,0.587105,0.316909,2.152221,0.682058,9.380952,0.736821,7.384898,0.438137,0.168775,0.126618,0.392406,0.124357,10.206714,17.284701,-0.020092
4,AOS,2019,Industrials,1.957469,2.352772,0.487932,0.595173,0.545062,0.834653,0.454938,42.281818,0.978646,5.076675,0.394527,0.123634,0.155412,0.221982,0.120994,2.278726,25.856549,0.105766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,ZION,2022,Financial Services,0.023772,0.023772,0.023772,0.017499,0.054643,17.300634,0.945357,5.227027,0.0352,8.165803,-0.290609,0.287754,0.306789,0.185367,0.010129,NaN,NaN,-0.174561
1956,ZTS,2019,Healthcare,2.629014,3.409745,1.069767,0.993909,0.23456,3.263294,0.76544,9.049327,0.542226,5.764273,0.681789,0.239617,0.322364,0.553914,0.129926,3.137419,46.805992,0.079989
1957,ZTS,2020,Healthcare,3.046544,3.796774,1.659908,0.979724,0.276949,2.609711,0.722757,9.822511,0.490484,6.589339,0.691835,0.245393,0.339925,0.434598,0.120362,3.44478,70.840508,-0.395146
1958,ZTS,2021,Healthcare,3.856427,4.926544,1.937674,1.231497,0.326835,2.059432,0.673094,12.513393,0.559424,6.863195,0.703832,0.26196,0.360468,0.448382,0.146547,4.29475,38.53542,0.470060


In [ ]:
yearly.columns.values

array(['periodType', 'currencyCode', 'AccountsPayable',
       'AccountsReceivable', 'AccumulatedDepreciation',
       'AllowanceForDoubtfulAccountsReceivable',
       'BuildingsAndImprovements', 'CapitalLeaseObligations',
       'CapitalStock', 'CashAndCashEquivalents',
       'CashCashEquivalentsAndShortTermInvestments', 'CommonStock',
       'CommonStockEquity', 'ConstructionInProgress',
       'CurrentAccruedExpenses', 'CurrentAssets', 'CurrentDebt',
       'CurrentDebtAndCapitalLeaseObligation', 'CurrentLiabilities',
       'DividendsPayable', 'EmployeeBenefits', 'FinishedGoods',
       'GainsLossesNotAffectingRetainedEarnings', 'Goodwill',
       'GoodwillAndOtherIntangibleAssets', 'GrossAccountsReceivable',
       'GrossPPE', 'IncomeTaxPayable', 'Inventory', 'InvestedCapital',
       'InvestmentsAndAdvances', 'LandAndImprovements',
       'LongTermCapitalLeaseObligation', 'LongTermDebt',
       'LongTermDebtAndCapitalLeaseObligation',
       'MachineryFurnitureEquipment', 'Minor

In [10]:
ADP = yq.Ticker('KMX')
balsheet = ADP.balance_sheet()
balsheet

'Balance Sheet data unavailable for KMX'

In [ ]:
incomeState = ADP.income_statement()
incomeState.columns.values

array(['asOfDate', 'periodType', 'currencyCode', 'BasicAverageShares',
       'BasicEPS', 'DilutedAverageShares', 'DilutedEPS',
       'DilutedNIAvailtoComStockholders', 'GainOnSaleOfSecurity',
       'GeneralAndAdministrativeExpense', 'InterestExpense',
       'InterestIncome', 'NetIncome', 'NetIncomeCommonStockholders',
       'NetIncomeContinuousOperations',
       'NetIncomeFromContinuingAndDiscontinuedOperation',
       'NetIncomeFromContinuingOperationNetMinorityInterest',
       'NetIncomeIncludingNoncontrollingInterests', 'NetInterestIncome',
       'NormalizedIncome', 'OperatingRevenue', 'OtherGandA',
       'OtherunderPreferredStockDividend', 'PreferredStockDividends',
       'PretaxIncome', 'ReconciledDepreciation', 'SalariesAndWages',
       'SellingAndMarketingExpense', 'SellingGeneralAndAdministration',
       'TaxEffectOfUnusualItems', 'TaxProvision', 'TaxRateForCalcs',
       'TotalRevenue'], dtype=object)

In [ ]:
yearly.columns.values

array(['periodType', 'currencyCode', 'AccumulatedDepreciation',
       'AdditionalPaidInCapital', 'AvailableForSaleSecurities',
       'BuildingsAndImprovements', 'CapitalStock',
       'CashAndCashEquivalents', 'CashFinancial', 'CommonStock',
       'CommonStockEquity', 'CurrentAccruedExpenses', 'CurrentDebt',
       'CurrentDebtAndCapitalLeaseObligation', 'CurrentNotesPayable',
       'GainsLossesNotAffectingRetainedEarnings', 'Goodwill',
       'GoodwillAndOtherIntangibleAssets', 'GrossPPE', 'InvestedCapital',
       'InvestmentsAndAdvances', 'LandAndImprovements', 'LongTermDebt',
       'LongTermDebtAndCapitalLeaseObligation', 'LongTermProvisions',
       'MachineryFurnitureEquipment', 'NetDebt', 'NetPPE',
       'NetTangibleAssets', 'OrdinarySharesNumber',
       'OtherCurrentBorrowings', 'OtherIntangibleAssets',
       'OtherReceivables', 'OtherShortTermInvestments',
       'PayablesAndAccruedExpenses', 'PreferredStock',
       'PreferredStockEquity', 'Receivables', 'RetainedEarn

In [ ]:
balance_sheet

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AllowanceForDoubtfulAccountsReceivable,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,CashEquivalents,CashFinancial,CommonStock,CommonStockEquity,CurrentAccruedExpenses,CurrentAssets,CurrentDebt,CurrentDebtAndCapitalLeaseObligation,CurrentLiabilities,EmployeeBenefits,FinishedGoods,GainsLossesNotAffectingRetainedEarnings,GrossAccountsReceivable,GrossPPE,HeldToMaturitySecurities,Inventory,InvestedCapital,InvestmentinFinancialAssets,InvestmentsAndAdvances,LongTermDebt,LongTermDebtAndCapitalLeaseObligation,LongTermEquityInvestment,LongTermProvisions,MinorityInterest,NetDebt,NetPPE,NetTangibleAssets,NonCurrentAccountsReceivable,NonCurrentDeferredLiabilities,NonCurrentDeferredTaxesLiabilities,NonCurrentNoteReceivables,OrdinarySharesNumber,OtherCurrentAssets,OtherInvestments,OtherNonCurrentAssets,OtherNonCurrentLiabilities,OtherProperties,OtherShortTermInvestments,Payables,PayablesAndAccruedExpenses,PensionandOtherPostRetirementBenefitPlansCurrent,PrepaidAssets,RawMaterials,Receivables,RetainedEarnings,ShareIssued,StockholdersEquity,TangibleBookValue,TotalAssets,TotalCapitalization,TotalDebt,TotalEquityGrossMinorityInterest,TotalLiabilitiesNetMinorityInterest,TotalNonCurrentAssets,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalTaxPayable,TreasurySharesNumber,TreasuryStock,WorkingCapital
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
COP,2019-12-31,12M,USD,3.200000e+09,3.401000e+09,-5.547700e+10,4.698300e+10,-13000000.0,18000000.0,5.088000e+09,1.022700e+10,2.846000e+09,2.242000e+09,18000000.0,3.498100e+10,2.045000e+09,1.691300e+10,1.050000e+08,1.050000e+08,7.043000e+09,1.781000e+09,472000000.0,-5.357000e+09,3.414000e+09,9.774600e+10,133000000.0,1.026000e+09,4.987600e+10,133000000.0,8.444000e+09,1.479000e+10,1.479000e+10,8.234000e+09,5.352000e+09,69000000.0,9.807000e+09,4.226900e+10,3.498100e+10,243000000.0,4.634000e+09,4.634000e+09,219000000.0,1.084868e+09,NaN,77000000.0,2.426000e+09,1.864000e+09,9.774600e+10,5.139000e+09,4.230000e+09,6.275000e+09,663000000.0,2.259000e+09,554000000.0,3.401000e+09,3.974200e+10,1.795652e+09,3.498100e+10,3.498100e+10,7.051400e+10,4.977100e+10,1.489500e+10,3.505000e+10,3.546400e+10,5.360100e+10,2.842100e+10,1.030000e+09,710783814.0,4.640500e+10,9.870000e+09
COP,2020-12-31,12M,USD,2.698000e+09,2.754000e+09,-6.221300e+10,4.713300e+10,-4000000.0,18000000.0,2.991000e+09,7.856000e+09,1.261000e+09,1.730000e+09,18000000.0,2.984900e+10,1.121000e+09,1.206600e+10,6.190000e+08,6.190000e+08,5.366000e+09,1.697000e+09,461000000.0,-5.218000e+09,2.758000e+09,1.021060e+11,217000000.0,1.002000e+09,4.521800e+10,217000000.0,7.880000e+09,1.475000e+10,1.475000e+10,7.596000e+09,5.430000e+09,0.0,1.237800e+10,3.989300e+10,2.984900e+10,137000000.0,3.747000e+09,3.747000e+09,114000000.0,1.068042e+09,NaN,67000000.0,2.528000e+09,1.779000e+09,1.021060e+11,4.865000e+09,3.018000e+09,4.139000e+09,608000000.0,4.540000e+08,541000000.0,2.754000e+09,3.521300e+10,1.798844e+09,2.984900e+10,2.984900e+10,6.261800e+10,4.459900e+10,1.536900e+10,2.984900e+10,3.276900e+10,5.055200e+10,2.740300e+10,3.200000e+08,730802089.0,4.729700e+10,6.700000e+09
COP,2021-12-31,12M,USD,5.025000e+09,6.670000e+09,-6.473500e+10,6.058100e+10,-2000000.0,21000000.0,5.028000e+09,6.591000e+09,2.804000e+09,2.224000e+09,21000000.0,4.540600e+10,2.179000e+09,1.605000e+10,1.200000e+09,1.200000e+09,1.202100e+10,1.153000e+09,647000000.0,-4.950000e+09,6.672000e+09,1.296460e+11,248000000.0,1.208000e+09,6.534000e+10,248000000.0,7.015000e+09,1.873400e+10,1.873400e+10,6.701000e+09,5.754000e+09,0.0,1.490600e+10,6.491100e+10,4.540600e+10,98000000.0,6.179000e+09,6.179000e+09,0.0,1.302243e+09,NaN,66000000.0,2.587000e+09,1.414000e+09,1.296460e+11,1.563000e+09,7.887000e+09,1.006600e+10,755000000.0,1.581000e+09,561000000.0,6.670000e+09,4.067400e+10,2.091563e+09,4.540600e+10,4.540600e+10,9.066100e+10,6.414000e+10,